In [ ]:
import os
import pandas as pd
import numpy as np
import preprocessing as proc
import some_functions as func
import randomforest as rf
from pandas.api.types import CategoricalDtype
import seaborn as sns
from matplotlib import pyplot as plt
from scipy import stats
import joblib

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler, Normalizer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor 
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score, confusion_matrix
from sklearn.metrics import pairwise_distances


%matplotlib inline

plt.style.use('seaborn-ticks')
sns.set_style('ticks')
plt.rcParams['figure.figsize'] = (8, 8)
plt.rcParams['axes.titlesize'] = 22
plt.rcParams['axes.labelsize'] = 22
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20

pd.options.display.max_columns = 1000

DATA_PATH = '../cell-profiler/measurements'
DATA_PATH2 = '../datasets/'
SUFFIX = 'BandC_'
CYTOPLASM = True
ZERNIKE = True
BIOMARKERS = True
NEIGHBOURS = True

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

** Import data **

In [ ]:
measurements = proc.import_cell_data(data_path=DATA_PATH, suffix=SUFFIX, 
                                     cytoplasm=CYTOPLASM, biomarkers=BIOMARKERS, neighbours=NEIGHBOURS)

** Clean data **

In [ ]:
measurements = proc.clean_data(measurements)

** Exclude poor quality images **

In [ ]:
to_exclude = ['0.2-B-A2-2',
              '0.2-B-A3-2',
              '0.2-C-B5-5',
              '2.0-B-A1-2', 
              '2.0-B-A2-4',
              '2.0-B-A3-2',
              '2.0-C-B4-1',
              '2.0-C-B4-3',
              '2.0-C-B4-4',
              '2.0-C-B4-5', 
              '16.0-C-B5-1', '16.0-C-B5-2', '16.0-C-B5-4', '16.0-C-B5-6', '16.0-C-B5-7',
              '64.0-C-B5-3']

for image in to_exclude:
    measurements.drop(measurements[measurements.image == image].index, inplace=True)

measurements.reset_index(drop=True, inplace=True)
    
print("Excluding images with poor quality from the analysis. The dataset contains {} cells.\n"
      .format(measurements.shape[0]))

** CTCF**

In [ ]:
# bg = proc.load_data("gain_Background.csv", DATA_PATH2)
# bg.bg_mean = bg.bg_mean/65535
# bg.bg_median = bg.bg_median/65535
# bg.bg_std = bg.bg_std/65535

# bg.head()

In [ ]:
# def ctcf(a, b, image, channel):
#     c = np.asscalar(bg[(bg.label == image) & (bg.channel == channel)].bg_median)
#     return a - b*c

# ctcf_dapi = []
# # ctcf_wga_nucl = []
# ctcf_wga = []
# ctcf_ker = []
# ctcf_vim = []

# for ind, row in measurements.iterrows():
#     # CTCF DAPI in nucleus
#     ctcf_dapi.append(ctcf(row.integratedintensity_dapi, 
#                           row.area_nucl, 
#                           row.image, 
#                           'DAPI'))
# #     # CTCF WGA in nucleus
# #     ctcf_wga_nucl.append(ctcf(row.integratedintensity_wga_nucl, 
# #                               row.area_nucl, 
# #                               row.image,
# #                               'WGA'))
#     # CTCF WGA in cell
#     ctcf_wga.append(ctcf(row.integratedintensity_wga, 
#                          row.area_cell, 
#                          row.image,
#                          'WGA'))
#     # CTCF Ker in cell
#     ctcf_ker.append(ctcf(row.integratedintensity_ker, 
#                          row.area_cell, 
#                          row.image,
#                          'Ker'))
#     # CTCF Vim in cell
#     ctcf_vim.append(ctcf(row.integratedintensity_vim, 
#                          row.area_cell, 
#                          row.image,
#                          'Vim'))

                     
# measurements['ctcf_dapi'] = ctcf_dapi
# # measurements['ctcf_wga_nucl'] = ctcf_wga_nucl
# measurements['ctcf_wga'] = ctcf_wga
# measurements['ctcf_ker'] = ctcf_ker
# measurements['ctcf_vim'] = ctcf_vim

** Feature engineering **

In [ ]:
measurements = proc.nc_ratio(measurements)
measurements = proc.cv_ratio(measurements, 
                             col1='integratedintensity_ker', 
                             col2='integratedintensity_vim')
measurements = proc.frac_on_edge(measurements, 'ecad')
measurements = proc.frac_on_edge(measurements, 'vim')
measurements = proc.frac_on_edge(measurements, 'ker')

** Feature selection **

In [ ]:
measurements.drop(['minferetdiameter_cyto', 'maxferetdiameter_cyto'], axis=1, inplace=True)
if select_features:
    measurements = proc.select_features(measurements, filename=selected_features)
    print("The dataset has {} features after feature selection.\n".format(measurements.shape[1]))

** Group features **

In [1]:
# Lists of column names
meta_cols = measurements.select_dtypes(include=['object', 'category']).columns
feature_cols = measurements.select_dtypes(include=[np.number]).columns
biom_cols = [col for col in measurements.columns if 'ker' in col or 'vim' in col or 'ecad' in col]
biom_cols.append('cvratio')
biom_cols.append('log_cvratio')
morph_cols = [col for col in feature_cols if col not in biom_cols]

print("{} columns in total:\n \
{} columns containing metadata, meta_cols,\n \
{} all features, feature_cols:\n \
{} biomarkers, biom_cols,\n \
{} morphology, morph_cols".format(measurements.columns.size, 
                      meta_cols.size, feature_cols.size, 
                      len(biom_cols), len(morph_cols)))

NameError: name 'measurements' is not defined

In [ ]:
geom_cols = [col for col in morph_cols if 'dapi' not in col and 'wga' not in col]
cell_cols = [col for col in morph_cols if 'cell' in col]
nucl_cols = [col for col in morph_cols if 'nucl' in col]
cyto_cols = [col for col in morph_cols if 'cyto' in col]
print("{} geometric measurements, geom_cols,\n \
{} cellular measurements, cell_cols,\n \
{} nuclear measurements, nucl_cols,\n \
{} cytoplasmic measurements, cyto_cols".format(len(geom_cols), len(cell_cols), len(nucl_cols), len(cyto_cols)))

###### Separate clusters at 64 kPa

In [ ]:
clumped = measurements.loc[(measurements.image == '64.0-B-A1-1') | 
                           (measurements.image == '64.0-B-A1-2') |
                           (measurements.image == '64.0-C-B5-1') |
                           (measurements.image == '64.0-C-B5-2') |
                           (measurements.image == '64.0-C-B6-1') |
                           (measurements.image == '64.0-C-B6-2'), :].reset_index(drop=True)

In [ ]:
to_exclude = clumped.image.unique()

measurements_red = measurements.copy()

for image in to_exclude:
    measurements_red.drop(measurements_red[measurements_red.image == image].index, inplace=True)

measurements_red.reset_index(drop=True, inplace=True)

In [ ]:
print("\n{} clumped cells excluded from the main dataser. The final dataset contains {} cells."
      .format(clumped.shape[0], measurements_red.shape[0]))

###### Exclude 0.5kPa and 8kPa

In [ ]:
# measurements_5 = measurements_red.drop(measurements_red[(measurements_red.stiffness == '0.5') | 
#                                                         (measurements_red.stiffness == '8.0')
#                                                        ].index
#                                       ).reset_index(drop=True)

# stiff_type = CategoricalDtype(categories=['0.2','2.0', '16.0', '32.0', '64.0'], ordered=True)
# measurements_5.stiffness = measurements_5.stiffness.astype(stiff_type)

# measurements_5.shape

###### Simple undersampling

In [ ]:
# measurements_b = proc.undersample(measurements_5, 50)

# stiff_type = CategoricalDtype(categories=['0.2','2.0', '16.0', '32.0', '64.0'], ordered=True)
# measurements_b.stiffness = measurements_b.stiffness.astype(stiff_type)